In [0]:
#set random seeds to get reproducible results
import numpy as np
np.random.seed(1)
tf.random.set_seed(2)

def create_model(learning_rate):
  model = applications.InceptionV3(include_top=False, weights=None, input_shape=image_size+(3,))
  #Using pretrained
  for layer in model.layers:
    layer.trainable=False
  #Adding Layers at the top
  top = model.output
  top = GlobalAveragePooling2D(data_format='channels_last', name='avg_pool')(top)

  #Adding Classification layer with softmax activation function(needed for categorical predictions)
  predictor = Dense(120 ,activation='softmax', name='predictor')(top)
  model = Model(inputs=model.input, outputs=predictor)
  model.load_weights('Initial weights.h5')
  model.compile(optimizer=optimizers.Adam(learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#install scikit optimization
!pip install scikit-optimize
import skopt
SEARCH_SPACE = [skopt.space.Real(0.0001, 0.01, name='learning_rate', prior='log_uniform')]

from skopt.utils import use_named_args
@use_named_args(SEARCH_SPACE)
def fitness(learning_rate):
  model = create_model(learning_rate)
  print('Running for early stopping')
  model.fit_generator(train_batches, epochs=5 , verbose=1, shuffle=True, validation_data=val_batches, callbacks=[early_stopping])
  model.load_weights('Initial weights.h5')
  print('Full training')
  training_results = model.fit_generator(train_val_batches, epochs=early_stopping.stopped_epoch , verbose=1, shuffle=True, validation_data=val_batches)
  print('')
  return training_results.history['val_accuracy'][-1]

In [0]:
result = skopt.forest_minimize(fitness, SEARCH_SPACE, n_calls=10)

In [0]:
skopt.plots.plot_evaluation(result)

In [0]:
skopt.plots.plot_objective(result)